### 以下我们将对已经收集好的验证码图片进行数据标注，并将其转换为训练集。

怎么标注？

- 为了提高模型的识别准确率，对数据的标注自然是越准确越好。但是，人工标注数据太耗费时间。所以，我们需要找别人做好的模型帮我们识别，并假装是我们手工标的（😄）。


`tesseract`是一个开源的OCR工具，可以识别各种语言的文字。python的`pytesseract`库可以调用`tesseract`命令行工具，实现对图片的文字识别。

首先，我们需要安装`tesseract`软件和`pytesseract`库。

安装`tesseract`软件，请参考github上的[readme文件](https://github.com/UB-Mannheim/tesseract/wiki)。
```
pip install pytesseract
```

In [82]:
from PIL import Image
# jupyter notebook环境下，取消注释并运行下一行代码，即可安装pytesseract库
# !pip install pytesseract 
import pytesseract
import numpy as np
import os

In [90]:
'''
关于os.walk()说明：
os.walk()是一个非常有用的函数，它可以用来遍历文件系统中的目录树。
返回值：生成器，分别是当前目录路径、所有子目录列表、所有文件列表。

关于生成器iterator：
生成器是一种特殊的迭代器，它可以生成一系列值，但只能在迭代时使用一次。
生成器的优点是可以节省内存，因为它一次只生成一个值，而不是生成整个序列。
'''
root, dirs, files = next(os.walk('./split'))# python的招牌语法糖
img = Image.open(os.path.join(root, files[1]))# 加载第一个图片
# img = Image.open('./captcha.gif') 
img

参数说明：
- lang="eng"：语言类型
- config:
    - psm 10：Page segmentation modes，可在命令行窗口输入tesseract --help-psm查看所有模式
    - tessedit_char_whitelist：顾名思义，只识别白名单中的字符

In [94]:
pytesseract.image_to_string(img, lang="eng", config='--psm 10 -c tessedit_char_whitelist=0123456789') # 识别验证码

'2\n'

### 第二阶段：转换成数据集格式

为了方便读取，我们把所有图片和对应的标签编码在一起，以一定的格式储存在文件中。

✋以什么样的格式储存呢？

In [14]:
import os

In [95]:
root, dir, files = next(os.walk("./split"))
for file in files[:10]:
    if file.endswith(".png"):
        img = Image.open(os.path.join(root, file))
        tag = pytesseract.image_to_string(
            img, lang="eng", config='--psm 10 -c tessedit_char_whitelist=0123456789')
        if tag and tag[0].isdigit():
            data = "".join([str(i//255) for i in tuple(img.tobytes())])
            print(file, tag[0], data)

1-1720445078.png 5 000000000000000000000000000000000000000000000000000000001111111000000111111111110000111111111111000111110000001100111100000000000111110000000000011110000000000011110001111100001111011111111100111111111111111011111000001111101111000000011110111000000001111011100000000111101110000000011110111100000011110011111000011111000111111111111000001111111111000000011111110000000000000000000000
1-1720445079.png 2 000000000000000000000000000000000000000000000000000000111111000000001111111111000001111111111100000110000011111000000000001111100000000000011110000000000011110000000000001111000000000001111000000000001111000000000001111000000000001110000000000011110000000000011110000000000011110000000000011110000000000011110000000000001111111111110000111111111111000011111111111100000000000000000000
1-1720445080.png 1 00000000000000000000000000000000000000000000000000000000001100000000001111110000000011111111000000001111111100000000110011110000000000001111000000000001111000000000000111100

👆很朴素的方法

## 整合

In [100]:
from PIL import Image
# !pip install pytesseract
import pytesseract
import numpy as np
import os

root, dir, files = next(os.walk("./split"))
with open("labels.txt", "w") as f:
    for file in files:
        if file.endswith(".png"):
            img = Image.open(os.path.join(root, file))
            tag = pytesseract.image_to_string(
                img, lang="eng", config='--psm 7 -c tessedit_char_whitelist=0123456789')
            if tag and tag[0].isdigit():
                f.write(str(tag[0]) + "".join([str(int(i>0)) for i in tuple(img.tobytes())]) + "\n")
                # f.write(str(tag[0]).encode() +img.tobytes()+ "\n".encode())

👇结果

In [98]:
with open('labels.txt', 'r') as f:
    raw = f.readlines()
raw[:10]

['5000000000000000000000000000000000000000000000000000000001111111000000111111111110000111111111111000111110000001100111100000000000111110000000000011110000000000011110001111100001111011111111100111111111111111011111000001111101111000000011110111000000001111011100000000111101110000000011110111100000011110011111000011111000111111111111000001111111111000000011111110000000000000000000000\n',
 '2000000000000000000000000000000000000000000000000000000111111000000001111111111000001111111111100000110000011111000000000001111100000000000011110000000000011110000000000001111000000000001111000000000001111000000000001111000000000001110000000000011110000000000011110000000000011110000000000011110000000000011110000000000001111111111110000111111111111000011111111111100000000000000000000\n',
 '1000000000000000000000000000000000000000000000000000000000011000000000011111100000000111111110000000011111111000000001100111100000000000011110000000000011110000000000001111000000000000111100000000000011110000000000